<a href="https://colab.research.google.com/github/lwy2107/GK_co_work/blob/master/%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이미지 처리를 하기위해서는 특징을 잘 추출 해내야한다. 그러므로 특징을 추출하기 위한 합성곱 작업을 해주어야하므로 우선 합성곱에 대해서 알아보자.